### ChatPDF
**智能文档助手，通过读取pdf内容回答相应问题**

功能：

- 加载专业领域的数据，如pdf

- 将数据存储到向量数据库中并检索出相关答案

- LLM根据检索出的答案重新整理回答，以获取更好的聊天体验


In [5]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"
os.environ["OPENAI_API_MODEL"] = "gpt-4-turbo"

### 加载pdf文件
Demo提供了一个pdf的训练数据，用以在专用领域回答问题

In [6]:
! pip install pypdf
! pip install langchain_community
! pip install -U langchain-text-splitters

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("test.pdf")
# 加载并分割
pages = loader.load_and_split()

print(len(pages))
print(pages)

32
[Document(page_content='易速鲜花集团 \n \n \n \n \n \n \n \n \n \n \n易速鲜花 服务中心  \n易速鲜花 股份有限公司  易\n速\n鲜\n花\n员\n工\n手\n册', metadata={'source': 'test.pdf', 'page': 0}), Document(page_content='易速鲜花集团 \n 1 董事长致辞  \n亲爱的同事：  \n您好！欢迎您加入 易速鲜花 旅游文化股份有限公司 ! \n我代表易速鲜花 对您的到来表示热烈的欢迎！并为公司拥有您\n这样优秀的员工而感到自豪和骄傲。 易速鲜花 将会因您的努力工作\n而稳步健康发展，将会因您的贡献而更加精彩。在此，我为您即将\n为易速鲜花 而付出的辛勤汗水表示诚挚的感谢！  \n易速鲜花 这个大家庭需要您、我、他每一位员工积极发扬“ 团\n结向上，完美无缺 ”的企业精神，以高度的主人翁责任感、使命感，\n与易速鲜花 同呼吸、共命 运，在各自的岗位上，勤奋敬业，尽职尽\n责，奋力拼搏。作为大家庭的一员，同仁之间应默契配合，相互接\n纳，取长补短，共同奋进，同舟共济。这样，我们就可以成为一个\n坚强的战斗堡垒， 在创 易速鲜花 发展的道路上披荆斩棘， 乘风破浪，\n无往不胜，所向披靡。最终以一流的服务、一流的管理、一流的信\n誉让我们的 易速鲜花 成为全国的一颗明珠。  \n希望各位以本手册为指南，共创 易速鲜花 美好明天 ! \n最后，诚挚地祝愿大家在公司工作愉快，前程似锦 ! \n \n \n                    签名：', metadata={'source': 'test.pdf', 'page': 1}), Document(page_content='易速鲜花集团 \n 2 易速鲜花 晨会宣言  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n今天 \n我开始新的生活  \n我要用全身心的爱迎接今天  \n我会用我全部的热情  \n关爱我的同事  \n热爱我们的公司  \n服务于我们的客户  \n忠于我们的工作  \n在努力工作中充实、完善，并超越自我  \n每天进步一点点  \n今天 \n我就付诸于行动！', metadata={'sou

### 向量化并存储

**将数据向量话并存储到Chroma中**

In [3]:
! pip install langchain-chroma

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 将之前分割的文档加载到Chroma中
db = Chroma.from_documents(pages, OpenAIEmbeddings())
print(db)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

### 在向量存储中查找答案

在存储中有多种查询方式

#### Similarity search（最近相似度搜索）

In [1]:
query = "去应聘需要提供什么资料"
docs = db.similarity_search(query)
print(docs)
print(docs[0].page_content) # 查询到的文档片段

NameError: name 'db' is not defined

#### MultiQueryRetriever（多重查询检索）

**MultiQueryRetriever 通过使用 LLM 为给定的用户输入查询从不同的角度生成多个查询，从而自动化提示调优过程。**

**对于每个查询，它检索一组相关文档，并在所有查询中使用唯一的联合以获得更大的一组可能相关的文档。**

**通过对同一个问题生成多个透视图，MultiQueryRetriever 可能能够克服基于距离的检索的一些局限性，并获得更丰富的结果集。**

In [12]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "员工的服务真理是什么?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs = retriever_from_llm.invoke(question)
print(len(unique_docs))
print(unique_docs)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. 什么是员工服务的真理？', '2. 员工服务的真理有哪些？', '3. 如何定义员工服务的真理？']


3
[Document(page_content='易速鲜花集团 \n 11 8、员工不得冷落顶撞、刁难顾客，接待顾客时应一视同仁，不得\n对顾客评头论足，做到文明礼貌待客。遇到顾客投诉应及时安抚顾\n客，第一时间忙住解决，不能解决的报部门主管处理。  \n9、每一位员工必须记住两条服务真理：  \n第一条：顾客永远是对的。  \n第二条：如果顾客错了，请参照第一条。  \n10、员工如与顾客发生争吵，罚款 50--100元，动手打顾客者，直\n接开除并追究损害公司形象的经济责任，情节严重者交司法机关依\n法处理。  \n11、讲究卫生，不随地吐痰，不乱扔杂物，不得在禁烟场所吸烟，\n保持工作环境清洁。  \n12、员工之间要团结协作、顾全大局，不得挑拨离间、制造事端，\n不扯皮推诿。  \n13、进入他人办公室前应先敲门，经同意后方可进入。离开时应随\n手关门。  \n14、要节约用水、用电、及相关办公品。  \n15、员工对相关工作或他人意见持有异议，可直接同上级主管反\n映，但不得终止工作；如确有问题，且在反映后不能得到及时解决\n的，可越级或向相关部门反映。  \n16、员工不得接受商户或顾客赠送的钱物等。  \n17、员工在公司遇见顾客及上级时，应主动侧身让路并微笑行注目\n礼。', metadata={'page': 11, 'source': 'test.pdf'}), Document(page_content='易速鲜花集团 \n 27 ④ 与顾客发生顶撞、争吵者；  \n⑤ 擅离岗位者；  \n⑥ 装病、骗取病假者；  \n⑦ 私自动用公司或损坏园区安全消防设施者；  \n⑧ 违反操作规程且造成一定后果者；  \n⑨ 上班时间使用电脑做与工作无关的事，看与本岗位无关的书\n报，所在办公室在场负责人及当事人同样受处罚；  \n4、员工出现下列情况之一者，除扣除 20-40分，公司还将予以惩处，\n并视情节轻重给予警告、开除等处分或予以辞退、解除劳动合同，并\n追究其相应责任：  \n① 污辱、谩骂、殴打顾客或同事者；  \n② 偷盗公司或他人钱物者；  \n③ 贪污挪用公款或受贿、索贿者；  \n④ 服务态度恶劣、造成顾客重大投诉，严重损坏公司声誉者；  \n⑤ 侵占货款、公物者；  \n⑥ 玩忽职守，造成公司物、财产重大损失者；  \n⑦ 无故旷

### LLM生成答案
将从向量数据库中检索到的片段提供给大模型，生成最终回答

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI(model="gpt-4-turbo")

template = [
     ("system","你是一个专门处理文档的秘书,你从不暴露自己是AI的身份,你会根据下面提供的上下文内容来继续回答问题.\n 上下文内容\n {context} \n"),
     ("human","你好！"),
     ("ai","你好"),
     ("human","{question}"),
]

prompt = ChatPromptTemplate.from_messages(template)
messages = prompt.format_messages(context=docs, question=query)

chat.invoke(messages)

AIMessage(content='应聘时需要提供以下资料：\n\n1. 本人如实填写的《入职申请表》一份；\n2. 近期免冠标准照片四张；\n3. 身份证复印件（需验明正件）；\n4. 学历证明（包括毕业证书、学位证书及验证证明）；\n5. 职称证书和资格证书；\n6. 与原单位解除劳动关系的证明文件；\n7. 公司指定医院的体检证明；\n8. 人力资源中心要求提供的其他资料。\n\n请确保这些文件齐全并符合要求，以便顺利完成应聘流程。', response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 1973, 'total_tokens': 2163}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_3450ce39d5', 'finish_reason': 'stop', 'logprobs': None}, id='run-24e94d74-c55f-47f4-bd57-62a50bd42921-0')